##PHASE 1: DATA COLLECTING

###1.1 INITIAL DATA EXPLORATION

In [1]:
pip install pandas numpy matplotlib seaborn scikit-learn

In [2]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount("/content/drive")

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/potable water/water_quality_potability.csv")

Mounted at /content/drive


#PHASE 3: PREPROCESSING

##3.1 LOAD DATA

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
import joblib
import warnings
warnings.filterwarnings("ignore")

# ============================================================================
# LOAD DATA
# ============================================================================
df = pd.read_csv("/content/drive/MyDrive/potable water/water_quality_potability.csv")

print(f" Dataset shape: {df.shape}")
print(f" Features: {df.shape[1] - 1}")
print(f" Target: Potability")

# Separate features and target
X = df.drop("Potability", axis=1)
y = df["Potability"]

# Single split - langsung train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f" Training set: {X_train.shape}")
print(f" Test set: {X_test.shape}")

 Dataset shape: (10000, 10)
 Features: 9
 Target: Potability
 Training set: (8000, 9)
 Test set: (2000, 9)


##3.2 HANDLE MISSING VALUE

In [4]:
imputer = KNNImputer(n_neighbors=5)
X_train_imputed = pd.DataFrame(
    imputer.fit_transform(X_train),
    columns=X_train.columns
)
X_test_imputed = pd.DataFrame(
    imputer.transform(X_test),
    columns=X_test.columns
)
print(f" Missing values handled")

# Save
joblib.dump(imputer, "/content/drive/MyDrive/potable water/imputer.pkl")

 Missing values handled


['/content/drive/MyDrive/potable water/imputer.pkl']

## 3.3 FEATURE ENGINEERING

In [5]:
import numpy as np
import pandas as pd

print("="*80)
print("FEATURE ENGINEERING")
print("="*80)

# ========================================
# STEP 1: CREATE DATA-DRIVEN FEATURES
# ========================================

def create_water_quality_features(X):
    """
    Parameters: X : DataFrame
    Preprocessed data (setelah imputation untuk missing value, sebelum handling outliers)
    Returns: X_fe : Data original +  feature baru
    """
    X_fe = X.copy()

    # -----------------
    # 1. RATIO FEATURES
    # -----------------

    # pH to hardness ratio (acidity vs mineral content)
    # X_fe["ph_hardness_ratio"] = X_fe["ph"] / (X_fe["Hardness"] + 1e-6)

    # Solids to conductivity ratio (dissolved matter)
    X_fe["solids_conductivity_ratio"] = X_fe["Solids"] / (X_fe["Conductivity"] + 1e-6)

    # Chloramines to turbidity ratio (disinfection effectiveness)
    # X_fe["chloramines_turbidity_ratio"] = X_fe["Chloramines"] / (X_fe["Turbidity"] + 1e-6)

    # Organic carbon to trihalomethanes ratio (contamination indicators)
    X_fe["organic_thm_ratio"] = X_fe["Organic_carbon"] / (X_fe["Trihalomethanes"] + 1e-6)

    # -----------------
    # 2. COMBINED FEATURES
    # -----------------

    # pH x Hardness (water chemistry interaction)
    # X_fe["ph_hardness_interaction"] = X_fe["ph"] * X_fe["Hardness"]

    # Chloramines x pH (disinfection effectiveness at different pH)
    X_fe["chloramines_ph_interaction"] = X_fe["Chloramines"] * X_fe["ph"]

    # Sulfate x Conductivity (ionic content indicator)
    X_fe["sulfate_conductivity_interaction"] = X_fe["Sulfate"] * X_fe["Conductivity"]

    # -----------------
    # 3. POLYNOMIAL FEATURES (non-linear relationships)
    # -----------------

    # pH squared (pH has non-linear effects on water quality)
    X_fe["ph_squared"] = X_fe["ph"] ** 2

    # Turbidity squared (clarity impact)
    X_fe["turbidity_squared"] = X_fe["Turbidity"] ** 2

    # Hardness squared (mineral content impact)
    # X_fe["hardness_squared"] = X_fe["Hardness"] ** 2

    # -----------------
    # 4. AGGREGATE FEATURES (summary statistics)
    # -----------------

    # Total dissolved substances indicator
    X_fe["total_minerals"] = X_fe["Hardness"] + X_fe["Sulfate"] + X_fe["Solids"]/1000

    # Total contamination indicator
    # X_fe["total_contamination"] = X_fe["Organic_carbon"] + X_fe["Trihalomethanes"] + X_fe["Turbidity"]

    # Chemical balance indicator
    X_fe["chemical_balance"] = X_fe["Chloramines"] + X_fe["Sulfate"] - X_fe["Organic_carbon"]

    # Water quality score
    X_fe["quality_score"] = (
        X_fe["Chloramines"] * 0.3 +  # disinfection
        (10 - abs(X_fe["ph"] - 7)) * 0.2 +  # pH neutrality
        (1 / (X_fe["Turbidity"] + 1)) * 0.3 +  # clarity
        (1 / (X_fe["Organic_carbon"] + 1)) * 0.2  # low contamination
    )

    print(f"\n Original features: {len(X.columns)}")
    print(f" New features created: {len(X_fe.columns) - len(X.columns)}")
    print(f" Total features: {len(X_fe.columns)}")

    return X_fe

print(f"Missing values in X_train_imputed: {X_train_imputed.isnull().sum().sum()}")
print(f"Missing values in X_test_imputed: {X_test_imputed.isnull().sum().sum()}")

if X_train_imputed.isnull().sum().sum() > 0:
    print(" Input ada missing values, harus cek imputation lagi")
else:
    print(" Input data clean (tidak ada missing values)")

# Create features for train and test
print("\n" + "="*80)
X_train_fe = create_water_quality_features(X_train_imputed)
X_test_fe = create_water_quality_features(X_test_imputed)

# ========================================
# STEP 2: ADD DOMAIN-BASED FEATURES
# ========================================

def add_domain_features(df):
    df_enhanced = df.copy()

    # COMPLIANCE FEATURES (Binary: 1 = compliant, 0 = not compliant)
    df_enhanced['pH_compliant'] = ((df['ph'] >= 6.5) & (df['ph'] <= 8.5)).astype(int)
    df_enhanced['Hardness_compliant'] = ((df['Hardness'] >= 60) & (df['Hardness'] <= 200)).astype(int)
    df_enhanced['Solids_compliant'] = (df['Solids'] < 500).astype(int)
    df_enhanced['Chloramines_compliant'] = (df['Chloramines'] <= 4).astype(int)
    df_enhanced['Sulfate_compliant'] = (df['Sulfate'] < 250).astype(int)
    df_enhanced['Conductivity_compliant'] = (df['Conductivity'] < 400).astype(int)
    df_enhanced['Organic_carbon_compliant'] = (df['Organic_carbon'] < 2).astype(int)
    df_enhanced['Trihalomethanes_compliant'] = (df['Trihalomethanes'] < 80).astype(int)
    df_enhanced['Turbidity_compliant'] = (df['Turbidity'] < 5).astype(int)

    # CATEGORICAL FEATURES
    df_enhanced['pH_category'] = pd.cut(df['ph'],
                                         bins=[0, 6.5, 8.5, 14],
                                         labels=['acidic', 'safe', 'alkaline'])

    df_enhanced['Hardness_category'] = pd.cut(df['Hardness'],
                                               bins=[0, 60, 200, np.inf],
                                               labels=['soft', 'optimal', 'hard'])

    df_enhanced['Turbidity_category'] = pd.cut(df['Turbidity'],
                                                bins=[0, 1, 5, np.inf],
                                                labels=['clear', 'acceptable', 'cloudy'])

    # DOMAIN ENGINEERED FEATURES
    compliance_cols = [col for col in df_enhanced.columns if col.endswith('_compliant')]
    df_enhanced['compliance_score'] = df_enhanced[compliance_cols].mean(axis=1)
    df_enhanced['violations_count'] = len(compliance_cols) - df_enhanced[compliance_cols].sum(axis=1)

    # Severity score (weighted by importance)
    df_enhanced['severity_score'] = (
        (1 - df_enhanced['pH_compliant']) * 2 +
        (1 - df_enhanced['Turbidity_compliant']) * 2 +
        (1 - df_enhanced['Chloramines_compliant']) * 1.5 +
        (1 - df_enhanced['Trihalomethanes_compliant']) * 1.5 +
        (1 - df_enhanced['Organic_carbon_compliant']) * 1 +
        (1 - df_enhanced['Hardness_compliant']) * 0.5 +
        (1 - df_enhanced['Solids_compliant']) * 1 +
        (1 - df_enhanced['Sulfate_compliant']) * 1 +
        (1 - df_enhanced['Conductivity_compliant']) * 1
    )

    print(f"Domain-based features created: {len(df_enhanced.columns) - len(df.columns)}")
    return df_enhanced

# Apply domain feature engineering
X_train_complete = add_domain_features(X_train_fe)
X_test_complete = add_domain_features(X_test_fe)

print(f"\n Total features now: {X_train_complete.shape[1]}")

# ========================================
# STEP 3: CATEGORIZE ALL FEATURES
# ========================================

# Original numerical features
original_numerical = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate',
                      'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

# Data-driven engineered features (all numerical, need scaling)
data_driven_features = [
    'solids_conductivity_ratio',
    'organic_thm_ratio', 'chloramines_ph_interaction',
    'sulfate_conductivity_interaction', 'ph_squared',
    'total_minerals', 'turbidity_squared',
    'chemical_balance', 'quality_score'
]

# Data WHO
compliance_features = [col for col in X_train_complete.columns if col.endswith('_compliant')]
categorical_features = [col for col in X_train_complete.columns if col.endswith('_category')]
domain_engineered = ['compliance_score', 'violations_count', 'severity_score']

print(f"\n Feature Breakdown:")
print(f"  Original Numerical:     {len(original_numerical):2d} features")
print(f"  Data-Driven Engineered: {len(data_driven_features):2d} features")
print(f"  Compliance Binary:      {len(compliance_features):2d} features")
print(f"  Categorical:            {len(categorical_features):2d} features")
print(f"  Domain Engineered:      {len(domain_engineered):2d} features")
print(f"  " + "-"*40)
print(f"  TOTAL:                  {len(original_numerical) + len(data_driven_features) + len(compliance_features) + len(categorical_features) + len(domain_engineered):2d} features")

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np

print("\n" + "="*100)
print("PREPROCESSING PIPELINE")
print("="*100)

# ========================================
# STEP 4: SCALING DAN ENCODING
# ========================================

preprocessor = ColumnTransformer(
    transformers=[
        ('original_num', StandardScaler(), original_numerical),
        ('data_driven', StandardScaler(), data_driven_features),
        ('compliance', 'passthrough', compliance_features),  # Already binary
        ('categorical', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_features),
        ('domain_eng', StandardScaler(), domain_engineered)
    ],
    remainder='drop'
)

# Fit and transform
X_train_processed = preprocessor.fit_transform(X_train_complete)
X_test_processed = preprocessor.transform(X_test_complete)

print(f"\n  Features before preprocessing: {X_train_complete.shape[1]}")
print(f"  Features after preprocessing:  {X_train_processed.shape[1]}")
print(f"\n  Breakdown:")
print(f"    - Original (scaled):        {len(original_numerical)}")
print(f"    - Data-driven (scaled):     {len(data_driven_features)}")
print(f"    - Compliance (binary):      {len(compliance_features)}")
print(f"    - Categorical (OHE):        {X_train_processed.shape[1] - len(original_numerical) - len(data_driven_features) - len(compliance_features) - len(domain_engineered)}")
print(f"    - Domain engineered (scaled): {len(domain_engineered)}")



FEATURE ENGINEERING
Missing values in X_train_imputed: 0
Missing values in X_test_imputed: 0
 Input data clean (tidak ada missing values)


 Original features: 9
 New features created: 9
 Total features: 18

 Original features: 9
 New features created: 9
 Total features: 18
Domain-based features created: 15
Domain-based features created: 15

 Total features now: 33

 Feature Breakdown:
  Original Numerical:      9 features
  Data-Driven Engineered:  9 features
  Compliance Binary:       9 features
  Categorical:             3 features
  Domain Engineered:       3 features
  ----------------------------------------
  TOTAL:                  33 features

PREPROCESSING PIPELINE

  Features before preprocessing: 33
  Features after preprocessing:  34

  Breakdown:
    - Original (scaled):        9
    - Data-driven (scaled):     9
    - Compliance (binary):      9
    - Categorical (OHE):        4
    - Domain engineered (scaled): 3


##3.3 HANDLE OUTLIERS

In [6]:
# ==============================================================================
# OUTLIER HANDLING DECISION
# ==============================================================================

print("\n" + "="*100)
print("OUTLIER HANDLING STRATEGY")
print("="*100)

# ==============================================================================
# VERIFY DATA INTEGRITY
# ==============================================================================

print("\n" + "-"*100)
print("Data Integrity Check")
print("-"*100)

# Reset index
X_train_complete = X_train_complete.reset_index(drop=True)
X_test_complete = X_test_complete.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

print(f"\n Training features (X_train_complete):  {X_train_complete.shape}")
print(f"Test features (X_test_complete):      {X_test_complete.shape}")
print(f"Training labels (y_train):            {y_train.shape}")
print(f"Test labels (y_test):                 {y_test.shape}")

print(f"\n Missing values in X_train_complete:   {X_train_complete.isnull().sum().sum()}")
print(f"Missing values in X_test_complete:    {X_test_complete.isnull().sum().sum()}")

print(f"\n Class distribution in y_train:")
print(y_train.value_counts(normalize=True).to_frame('percentage').round(4))

print(f"\n Class distribution in y_test:")
print(y_test.value_counts(normalize=True).to_frame('percentage').round(4))

# ==============================================================================
# OUTLIER STATISTICS (For Information Only)
# ==============================================================================

print("\n" + "-"*100)
print("Outlier Statistics (IQR Method - Information Only)")
print("-"*100)

# Original features for outlier detection
original_features = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate',
                     'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

def detect_outliers_iqr(df, column):
    """Detect outliers using IQR method"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), (len(outliers) / len(df)) * 100

print("\n Outliers detected (but KEPT in dataset):")
print(f"{'Feature':<25} {'Outlier Count':>15} {'Percentage':>12}")
print("-" * 55)

total_outlier_rows = 0
for feature in original_features:
    count, pct = detect_outliers_iqr(X_train_complete, feature)
    print(f"{feature:<25} {count:>15} {pct:>11.2f}%")

outlier_mask = pd.Series([False] * len(X_train_complete))
for feature in original_features:
    Q1 = X_train_complete[feature].quantile(0.25)
    Q3 = X_train_complete[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outlier_mask |= (X_train_complete[feature] < lower_bound) | (X_train_complete[feature] > upper_bound)

total_outlier_rows = outlier_mask.sum()
outlier_row_pct = (total_outlier_rows / len(X_train_complete)) * 100

print("-" * 55)
print(f"\n{'Total rows with outliers:':<25} {total_outlier_rows:>15} {outlier_row_pct:>11.2f}%")
print(f"{'Rows kept for training:':<25} {len(X_train_complete):>15} {'100.00%':>12}")

# ==============================================================================
# PREPROCESSED DATA STATUS
# ==============================================================================

print("\n" + "="*100)
print("PREPROCESSED DATA STATUS")
print("="*100)

print(f"\n  X_train_processed shape: {X_train_processed.shape}")
print(f"  X_test_processed shape:  {X_test_processed.shape}")
print(f"\n  Features breakdown:")
print(f"    - Before preprocessing: {X_train_complete.shape[1]} features")
print(f"    - After preprocessing:  {X_train_processed.shape[1]} features")
print(f"    - Samples in training:  {X_train_processed.shape[0]} samples")
print(f"    - Samples in test:      {X_test_processed.shape[0]} samples")

# ==============================================================================
# FINAL CONFIRMATION
# ==============================================================================

print("\n" + "="*100)
print(" DATA READY FOR MODELING")
print("="*100)

# Optional: Store dataset info untuk tracking
dataset_info = {
    'strategy': 'keep_all_outliers',
    'X_train_shape': X_train_processed.shape,
    'X_test_shape': X_test_processed.shape,
    'features_before_preprocessing': X_train_complete.shape[1],
    'features_after_preprocessing': X_train_processed.shape[1],
    'outlier_rows_in_train': total_outlier_rows,
    'outlier_percentage': outlier_row_pct,
    'total_samples_train': len(X_train_processed),
    'total_samples_test': len(X_test_processed)
}

print("\n Dataset Summary:")
for key, value in dataset_info.items():
    print(f"  {key}: {value}")




OUTLIER HANDLING STRATEGY

----------------------------------------------------------------------------------------------------
Data Integrity Check
----------------------------------------------------------------------------------------------------

 Training features (X_train_complete):  (8000, 33)
Test features (X_test_complete):      (2000, 33)
Training labels (y_train):            (8000,)
Test labels (y_test):                 (2000,)

 Missing values in X_train_complete:   0
Missing values in X_test_complete:    0

 Class distribution in y_train:
            percentage
Potability            
0                  0.5
1                  0.5

 Class distribution in y_test:
            percentage
Potability            
0                  0.5
1                  0.5

----------------------------------------------------------------------------------------------------
Outlier Statistics (IQR Method - Information Only)
------------------------------------------------------------------------

SMOTE

In [7]:
from imblearn.combine import SMOTETomek
from sklearn.utils.class_weight import compute_sample_weight

print("\n" + "="*100)
print("APPLYING SMOTE + TOMEK LINKS")
print("="*100)

print(f"Before resampling:")
print(f"  Class 0: {(y_train == 0).sum()}")
print(f"  Class 1: {(y_train == 1).sum()}")

smote_tomek = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train_processed, y_train)

print(f"\nAfter SMOTE+Tomek:")
print(f"  Class 0: {(y_train_resampled == 0).sum()}")
print(f"  Class 1: {(y_train_resampled == 1).sum()}")

# Update sample weights
sample_weights = compute_sample_weight('balanced', y=y_train_resampled)
scale_pos_weight = (y_train_resampled == 0).sum() / (y_train_resampled == 1).sum()


APPLYING SMOTE + TOMEK LINKS
Before resampling:
  Class 0: 4000
  Class 1: 4000

After SMOTE+Tomek:
  Class 0: 3702
  Class 1: 3702


##3.5 SAVE PREPROCESSED DATA

In [8]:
import joblib
import os
import pandas as pd
import numpy as np

print("="*80)
print("SAVING PREPROCESSED DATA")
print("="*80)

output_dir = "/content/drive/MyDrive/potable water/outputs"
os.makedirs(output_dir, exist_ok=True)

X_train_processed_df = pd.DataFrame(X_train_processed)
X_test_processed_df = pd.DataFrame(X_test_processed)

# Save training data
X_train_processed_df.to_csv(f"{output_dir}/X_train_processed.csv", index=False)
y_train.to_csv(f"{output_dir}/y_train_resampled.csv", index=False, header=True)

# Save test data
X_test_processed_df.to_csv(f"{output_dir}/X_test_processed.csv", index=False)
y_test.to_csv(f"{output_dir}/y_test_processed.csv", index=False, header=True)

# Save preprocessing objects
joblib.dump(imputer, f"{output_dir}/imputer.pkl")

feature_info = {
    "original_features": list(X_train_imputed.columns),
    "new_features": [col for col in X_train_fe.columns if col not in X_train_imputed.columns],
    "all_features": [f'feature_{i}' for i in range(X_train_processed.shape[1])], # Placeholder names
    "total_features": X_train_processed.shape[1]
}

import json
with open(f"{output_dir}/feature_info.json", "w") as f:
    json.dump(feature_info, f, indent=2)



SAVING PREPROCESSED DATA


#PHASE 4: MODEL

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import time
from datetime import datetime

# Models
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Evaluation
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix, roc_auc_score, roc_curve
)
from sklearn.utils.class_weight import compute_sample_weight

from sklearn.preprocessing import OneHotEncoder # Import OneHotEncoder for feature name extraction

import warnings
warnings.filterwarnings('ignore')

print("="*100)
print("BASELINE MODEL TRAINING - RANDOM FOREST")
print("="*100)

print(f"\n Training set X_train_processed: {X_train_processed.shape}")
print(f" Training set X_train_resampled: {X_train_resampled.shape}")
print(f" Test set: {X_test_processed.shape}")
print(f" Features: {X_train_resampled.shape[1]}")

print(f"\n Target distribution (y_train_resampled):")
print(f"    Class 0 (Not Potable): {(y_train_resampled == 0).sum()} ({(y_train_resampled == 0).sum()/len(y_train_resampled)*100:.1f}%)")
print(f"    Class 1 (Potable): {(y_train_resampled == 1).sum()} ({(y_train_resampled == 1).sum()/len(y_train_resampled)*100:.1f}%)")

print(f"\n Target distribution (y_test):")
print(f"  Class 0 (Not Potable): {(y_test == 0).sum():4d} ({(y_test == 0).sum()/len(y_test)*100:.1f}%)")
print(f"  Class 1 (Potable):     {(y_test == 1).sum():4d} ({(y_test == 1).sum()/len(y_test)*100:.1f}%)")

# ==============================================================================
# BASELINE MODEL
# ==============================================================================

sample_weights = compute_sample_weight('balanced', y=y_train_resampled)
scale_pos_weight = (y_train_resampled == 0).sum() / (y_train_resampled == 1).sum()

print(f"\nClass imbalance handling:")
print(f"  Scale pos weight (XGBoost) for resampled data: {scale_pos_weight:.2f}")
print(f"  Sample weights for resampled data calculated")

models = {
    'Random Forest': RandomForestClassifier(
        n_estimators=200,
        max_depth=15,
        min_samples_split=5,
        min_samples_leaf=2,
        max_features='sqrt',
        max_samples=0.8,             # Bootstrap sample size (80% dari data)
        random_state=42,
        n_jobs=-1,
        class_weight='balanced',
        oob_score=True
    ),
}

print("\n" + "="*100)
print("TRAINING BASELINE MODELS")
print("="*100)

# Store results
baseline_results = {}

# Cross-validation setup
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("\nTraining baseline models with 5-fold cross-validation")
print("-" * 100)

for name, model in models.items():
    print(f"\n {name}...")

    try:
        start_time = time.time()

        # Perform cross-validation
        print(f"  Running 5-fold cross-validation...")
        cv_scores = cross_val_score(
            model, X_train_resampled, y_train_resampled,
            cv=cv,
            scoring='roc_auc',
            n_jobs=-1
        )

        print(f"  CV ROC-AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std()*2:.4f})")
        print(f"  Individual folds: {[f'{score:.4f}' for score in cv_scores]}")

        # Train full training data
        if name in ['Gradient Boosting', 'AdaBoost']:
            model.fit(X_train_resampled, y_train_resampled, sample_weight=sample_weights)
        else:
            model.fit(X_train_resampled, y_train_resampled)

        train_time = time.time() - start_time

        if hasattr(model, 'oob_score_'):
            print(f"  OOB Score:      {model.oob_score_:.4f}")

        # Test set predictions
        y_pred = model.predict(X_test_processed)

        # ROC-AUC
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test_processed)[:, 1]
            test_roc_auc = roc_auc_score(y_test, y_pred_proba)
        else:
            y_pred_proba = None
            test_roc_auc = np.nan

        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        test_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        test_f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        # PER-CLASS METRICS
        test_precision_class0 = precision_score(y_test, y_pred, pos_label=0, zero_division=0)
        test_recall_class0 = recall_score(y_test, y_pred, pos_label=0, zero_division=0)
        test_f1_class0 = f1_score(y_test, y_pred, pos_label=0, zero_division=0)

        test_precision_class1 = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
        test_recall_class1 = recall_score(y_test, y_pred, pos_label=1, zero_division=0)
        test_f1_class1 = f1_score(y_test, y_pred, pos_label=1, zero_division=0)

        # Store results
        baseline_results[name] = {
            'cv_roc_auc_mean': cv_scores.mean(),
            'cv_roc_auc_std': cv_scores.std(),
            'cv_scores': cv_scores,
            'test_accuracy': test_accuracy,
            'test_precision': test_precision,
            'test_recall': test_recall,
            'test_f1': test_f1,
            'test_roc_auc': test_roc_auc,

            # Per-class metrics
            'precision_class0': test_precision_class0,
            'recall_class0': test_recall_class0,
            'f1_class0': test_f1_class0,
            'precision_class1': test_precision_class1,
            'recall_class1': test_recall_class1,
            'f1_class1': test_f1_class1,

            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba,
            'train_time': train_time,
            'model': model
        }

        print(f"  Test Accuracy:  {test_accuracy:.4f}")
        print(f"  Test ROC-AUC:   {test_roc_auc:.4f}")
        print(f"  Test F1-Score:  {test_f1:.4f}")
        print(f"  Class 1 Recall: {test_recall_class1:.4f} (Potable detection)")
        print(f"  Training Time:  {train_time:.2f}s")

    except Exception as e:
        print(f"  Error: {str(e)}")
        continue

# ==============================================================================
# RESULTS SUMMARY
# ==============================================================================
print("\n" + "="*100)
print(" RESULTS SUMMARY ")
print("="*100)

if baseline_results:
    results_df = pd.DataFrame({
        'Model': list(baseline_results.keys()),
        'CV ROC-AUC': [f"{baseline_results[m]['cv_roc_auc_mean']:.4f} \u00b1 {baseline_results[m]['cv_roc_auc_std']:.4f}"
                       for m in baseline_results],
        'Test Accuracy': [baseline_results[m]['test_accuracy'] for m in baseline_results],
        'Test Precision': [baseline_results[m]['test_precision'] for m in baseline_results],
        'Test Recall': [baseline_results[m]['test_recall'] for m in baseline_results],
        'Test F1': [baseline_results[m]['test_f1'] for m in baseline_results],
        'Test ROC-AUC': [baseline_results[m]['test_roc_auc'] for m in baseline_results],
        'Train Time (s)': [baseline_results[m]['train_time'] for m in baseline_results]
    })

    # Sort by Test ROC-AUC
    results_df = results_df.sort_values('Test ROC-AUC', ascending=False)

    print("\n", results_df.to_string(index=False))

    # Identify best model
    best_model_name = results_df.iloc[0]['Model']
    best_roc_auc = baseline_results[best_model_name]['test_roc_auc']
    best_cv_roc_auc = baseline_results[best_model_name]['cv_roc_auc_mean']

    print(f"\n Best Model: {best_model_name}")
    print(f"   CV ROC-AUC:   {best_cv_roc_auc:.4f}")
    print(f"   Test ROC-AUC: {best_roc_auc:.4f}")

    # ==============================================================================
    # DETAILED PER-CLASS RESULTS
    # ==============================================================================
    print("\n" + "="*100)
    print(" DETAILED PER-CLASS PERFORMANCE ")
    print("="*100)

    for idx, row in results_df.iterrows():
        model_name = row['Model']
        metrics = baseline_results[model_name]

        print(f"\n{'='*80}")
        print(f"{model_name}")
        print(f"{'='*80}")

        # Overall metrics
        print(f"\nOverall Metrics:")
        print(f"  CV ROC-AUC:     {metrics['cv_roc_auc_mean']:.4f} (+/- {metrics['cv_roc_auc_std']*2:.4f})")
        print(f"  Test Accuracy:  {metrics['test_accuracy']:.4f}")
        print(f"  Test F1:        {metrics['test_f1']:.4f}")
        print(f"  Test ROC-AUC:   {metrics['test_roc_auc']:.4f}")

        # Class-specific breakdown
        print(f"\nClass 0 (Not Potable):")
        print(f"  Precision: {metrics['precision_class0']:.4f}")
        print(f"  Recall:    {metrics['recall_class0']:.4f}")
        print(f"  F1-Score:  {metrics['f1_class0']:.4f}")

        print(f"\nClass 1 (Potable):")
        print(f"  Precision: {metrics['precision_class1']:.4f}")
        print(f"  Recall:    {metrics['recall_class1']:.4f}")
        print(f"  F1-Score:  {metrics['f1_class1']:.4f}")

        # Classification report
        print(f"\nClassification Report:")
        print(classification_report(y_test, metrics['y_pred'],
                                   target_names=['Not Potable', 'Potable'],
                                   digits=4))

    # ==============================================================================
    # PER-CLASS COMPARISON TABLE
    # ==============================================================================
    print("\n" + "="*100)
    print(" PER-CLASS F1-SCORE COMPARISON ")
    print("="*100)

    comparison_df = pd.DataFrame({
        'Model': list(baseline_results.keys()),
        'CV ROC-AUC': [f"{baseline_results[m]['cv_roc_auc_mean']:.4f}" for m in baseline_results],
        'Precision (0)': [f"{baseline_results[m]['precision_class0']:.4f}" for m in baseline_results],
        'Recall (0)': [f"{baseline_results[m]['recall_class0']:.4f}" for m in baseline_results],
        'F1 (0)': [f"{baseline_results[m]['f1_class0']:.4f}" for m in baseline_results],
        'Precision (1)': [f"{baseline_results[m]['precision_class1']:.4f}" for m in baseline_results],
        'Recall (1)': [f"{baseline_results[m]['recall_class1']:.4f}" for m in baseline_results],
        'F1 (1)': [f"{baseline_results[m]['f1_class1']:.4f}" for m in baseline_results],
        'Test ROC-AUC': [f"{baseline_results[m]['test_roc_auc']:.4f}" for m in baseline_results],
    })

    # Sort by Test ROC-AUC
    roc_scores = [baseline_results[m]['test_roc_auc'] for m in baseline_results]
    comparison_df['_sort'] = roc_scores
    comparison_df = comparison_df.sort_values('_sort', ascending=False)
    comparison_df = comparison_df.drop('_sort', axis=1)

    print("\n", comparison_df.to_string(index=False))

    # ==============================================================================
    # FEATURE IMPORTANCE ANALYSIS
    # ==============================================================================
    print("\n" + "="*100)
    print(" FEATURE IMPORTANCE ANALYSIS ")
    print("="*100)

    best_model = baseline_results[best_model_name]['model']

    if hasattr(best_model, 'feature_importances_'):

        def get_processed_feature_names(preprocessor, input_df_columns):
            output_feature_names = []
            for name, transformer, features_orig in preprocessor.transformers_:
                if transformer == 'passthrough':
                    output_feature_names.extend(features_orig)
                elif hasattr(transformer, 'get_feature_names_out'):
                    if isinstance(transformer, OneHotEncoder):
                        output_feature_names.extend(transformer.get_feature_names_out(features_orig))
                    else:
                        output_feature_names.extend(features_orig)
                else:
                    output_feature_names.extend(features_orig)
            return output_feature_names

        feature_names = get_processed_feature_names(preprocessor, X_train_complete.columns)

        # Create feature importance dataframe
        feature_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"\nTop 10 Most Important Features for {best_model_name}:")
        print(feature_importance.head(10).to_string(index=False))

        print(f"\nBottom 5 Least Important Features:")
        print(feature_importance.tail(5).to_string(index=False))

        # Summary stats
        print(f"\nFeature Importance Statistics:")
        print(f"  Mean importance: {feature_importance['importance'].mean():.6f}")
        print(f"  Median importance: {feature_importance['importance'].median():.6f}")
        print(f"  Features with >5% importance: {(feature_importance['importance'] > 0.05).sum()}")
        print(f"  Features with <1% importance: {(feature_importance['importance'] < 0.01).sum()}")

else:
    print("\nNo models were successfully trained. 'baseline_results' is empty.")

print("\n" + "="*100)
print("TRAINING COMPLETE")
print("="*100)


BASELINE MODEL TRAINING - RANDOM FOREST

 Training set X_train_processed: (8000, 34)
 Training set X_train_resampled: (7404, 34)
 Test set: (2000, 34)
 Features: 34

 Target distribution (y_train_resampled):
    Class 0 (Not Potable): 3702 (50.0%)
    Class 1 (Potable): 3702 (50.0%)

 Target distribution (y_test):
  Class 0 (Not Potable): 1000 (50.0%)
  Class 1 (Potable):     1000 (50.0%)

Class imbalance handling:
  Scale pos weight (XGBoost) for resampled data: 1.00
  Sample weights for resampled data calculated

TRAINING BASELINE MODELS

Training baseline models with 5-fold cross-validation
----------------------------------------------------------------------------------------------------

 Random Forest...
  Running 5-fold cross-validation...
  CV ROC-AUC: 0.9527 (+/- 0.0108)
  Individual folds: ['0.9614', '0.9531', '0.9518', '0.9528', '0.9443']
  OOB Score:      0.8790
  Test Accuracy:  0.8490
  Test ROC-AUC:   0.9345
  Test F1-Score:  0.8487
  Class 1 Recall: 0.8070 (Potable det

#PHASE 5: HYPERPARAMETER TUNING

In [11]:
# ==============================================================================
# HYPERPARAMETER TUNING - RANDOM FOREST
# ==============================================================================

print("\n" + "="*100)
print("HYPERPARAMETER TUNING - RANDOM FOREST")
print("="*100)

# Get the baseline model for reference
baseline_model = baseline_results['Random Forest']['model']
baseline_test_roc_auc = baseline_results['Random Forest']['test_roc_auc']
baseline_cv_roc_auc = baseline_results['Random Forest']['cv_roc_auc_mean']

print(f"\nBaseline Performance:")
print(f"  CV ROC-AUC:   {baseline_cv_roc_auc:.4f}")
print(f"  Test ROC-AUC: {baseline_test_roc_auc:.4f}")

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [150, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
    'max_samples': [0.7, 0.8]
}

print(f"\nParameter Grid:")
print(f"  n_estimators: {param_grid['n_estimators']}")
print(f"  max_depth: {param_grid['max_depth']}")
print(f"  min_samples_split: {param_grid['min_samples_split']}")
print(f"  min_samples_leaf: {param_grid['min_samples_leaf']}")
print(f"  max_features: {param_grid['max_features']}")
print(f"  max_samples: {param_grid['max_samples']}")

# Calculate total combinations
total_combinations = 1
for param_values in param_grid.values():
    total_combinations *= len(param_values)
print(f"\nTotal parameter combinations: {total_combinations}")
print(f"Total fits (5-fold CV): {total_combinations * 5}")

tuned_results = {}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(
        random_state=42,
        n_jobs=-1,
        class_weight='balanced',
        oob_score=True
    ),
    param_grid=param_grid,
    cv=cv,  # Using the same StratifiedKFold
    scoring='roc_auc',
    n_jobs=-1,
    verbose=2,
    return_train_score=True
)

# Perform grid search
start_time = time.time()
grid_search.fit(X_train_resampled, y_train_resampled)
tuning_time = time.time() - start_time

print(f"\nGrid Search completed in {tuning_time:.2f}s ({tuning_time/60:.2f} minutes)")

# ==============================================================================
# BEST PARAMETERS AND RESULTS
# ==============================================================================
print("\n" + "="*100)
print(" TUNING RESULTS ")
print("="*100)

print(f"\nBest Parameters Found:")
for param, value in grid_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest CV ROC-AUC: {grid_search.best_score_:.4f}")

# Get the best model
best_tuned_model = grid_search.best_estimator_

# ==============================================================================
# CALCULATE CV F1 SCORE
# ==============================================================================
print(f"\nCalculating CV F1 score...")
cv_f1_scores = cross_val_score(
    best_tuned_model,
    X_train_resampled,
    y_train_resampled,
    cv=cv,
    scoring='f1_weighted',
    n_jobs=-1
)
cv_f1_mean = cv_f1_scores.mean()
cv_f1_std = cv_f1_scores.std()

print(f"  CV F1 (mean ± std): {cv_f1_mean:.4f} ± {cv_f1_std:.4f}")

# ==============================================================================
# EVALUATE ON TEST SET
# ==============================================================================
print(f"\nEvaluating tuned model on test set...")
y_pred_tuned = best_tuned_model.predict(X_test_processed)
y_pred_proba_tuned = best_tuned_model.predict_proba(X_test_processed)[:, 1]

# Calculate metrics
tuned_test_accuracy = accuracy_score(y_test, y_pred_tuned)
tuned_test_precision = precision_score(y_test, y_pred_tuned, average='weighted', zero_division=0)
tuned_test_recall = recall_score(y_test, y_pred_tuned, average='weighted', zero_division=0)
tuned_test_f1 = f1_score(y_test, y_pred_tuned, average='weighted', zero_division=0)
tuned_test_roc_auc = roc_auc_score(y_test, y_pred_proba_tuned)

# Per-class metrics
tuned_precision_class0 = precision_score(y_test, y_pred_tuned, pos_label=0, zero_division=0)
tuned_recall_class0 = recall_score(y_test, y_pred_tuned, pos_label=0, zero_division=0)
tuned_f1_class0 = f1_score(y_test, y_pred_tuned, pos_label=0, zero_division=0)

tuned_precision_class1 = precision_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)
tuned_recall_class1 = recall_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)
tuned_f1_class1 = f1_score(y_test, y_pred_tuned, pos_label=1, zero_division=0)

# OOB Score
tuned_oob_score = best_tuned_model.oob_score_ if hasattr(best_tuned_model, 'oob_score_') else np.nan

# ==============================================================================
# FINAL SUMMARY
# ==============================================================================
tuned_results['Random Forest'] = {
    'model': best_tuned_model,
    'best_params': grid_search.best_params_,
    # CV metrics - consistent naming (no "_mean" suffix)
    'cv_roc_auc': grid_search.best_score_,
    'cv_f1': cv_f1_mean,
    # Test metrics
    'test_accuracy': tuned_test_accuracy,
    'test_precision': tuned_test_precision,
    'test_recall': tuned_test_recall,
    'test_f1': tuned_test_f1,
    'test_roc_auc': tuned_test_roc_auc,
    # Additional metrics
    'oob_score': tuned_oob_score,
    'precision_class0': tuned_precision_class0,
    'recall_class0': tuned_recall_class0,
    'f1_class0': tuned_f1_class0,
    'precision_class1': tuned_precision_class1,
    'recall_class1': tuned_recall_class1,
    'f1_class1': tuned_f1_class1,
    'training_time': tuning_time,
    'y_pred': y_pred_tuned,
    'y_pred_proba': y_pred_proba_tuned
}

print(f"\nTuned Model Performance:")
print(f"  CV ROC-AUC:     {grid_search.best_score_:.4f}")
print(f"  CV F1:          {cv_f1_mean:.4f}")
print(f"  Test Accuracy:  {tuned_test_accuracy:.4f}")
print(f"  Test Precision: {tuned_test_precision:.4f}")
print(f"  Test Recall:    {tuned_test_recall:.4f}")
print(f"  Test F1:        {tuned_test_f1:.4f}")
print(f"  Test ROC-AUC:   {tuned_test_roc_auc:.4f}")
print(f"  OOB Score:      {tuned_oob_score:.4f}")

# ==============================================================================
# BASELINE VS TUNED COMPARISON
# ==============================================================================
print("\n" + "="*100)
print(" BASELINE VS TUNED MODEL COMPARISON ")
print("="*100)

comparison_table = pd.DataFrame({
    'Metric': ['CV ROC-AUC', 'Test Accuracy', 'Test Precision', 'Test Recall',
               'Test F1', 'Test ROC-AUC', 'OOB Score'],
    'Baseline': [
        f"{baseline_cv_roc_auc:.4f}",
        f"{baseline_results['Random Forest']['test_accuracy']:.4f}",
        f"{baseline_results['Random Forest']['test_precision']:.4f}",
        f"{baseline_results['Random Forest']['test_recall']:.4f}",
        f"{baseline_results['Random Forest']['test_f1']:.4f}",
        f"{baseline_test_roc_auc:.4f}",
        f"{baseline_model.oob_score_:.4f}" if hasattr(baseline_model, 'oob_score_') else 'N/A'
    ],
    'Tuned': [
        f"{grid_search.best_score_:.4f}",
        f"{tuned_test_accuracy:.4f}",
        f"{tuned_test_precision:.4f}",
        f"{tuned_test_recall:.4f}",
        f"{tuned_test_f1:.4f}",
        f"{tuned_test_roc_auc:.4f}",
        f"{tuned_oob_score:.4f}" if not np.isnan(tuned_oob_score) else 'N/A'
    ],
    'Improvement': [
        f"{(grid_search.best_score_ - baseline_cv_roc_auc):.4f}",
        f"{(tuned_test_accuracy - baseline_results['Random Forest']['test_accuracy']):.4f}",
        f"{(tuned_test_precision - baseline_results['Random Forest']['test_precision']):.4f}",
        f"{(tuned_test_recall - baseline_results['Random Forest']['test_recall']):.4f}",
        f"{(tuned_test_f1 - baseline_results['Random Forest']['test_f1']):.4f}",
        f"{(tuned_test_roc_auc - baseline_test_roc_auc):.4f}",
        f"{(tuned_oob_score - baseline_model.oob_score_):.4f}" if hasattr(baseline_model, 'oob_score_') else 'N/A'
    ]
})

print("\n", comparison_table.to_string(index=False))

# ==============================================================================
# PER-CLASS PERFORMANCE COMPARISON
# ==============================================================================
print("\n" + "="*100)
print(" PER-CLASS PERFORMANCE COMPARISON ")
print("="*100)

per_class_comparison = pd.DataFrame({
    'Class': ['Not Potable (0)', 'Not Potable (0)', 'Not Potable (0)',
              'Potable (1)', 'Potable (1)', 'Potable (1)'],
    'Metric': ['Precision', 'Recall', 'F1-Score', 'Precision', 'Recall', 'F1-Score'],
    'Baseline': [
        f"{baseline_results['Random Forest']['precision_class0']:.4f}",
        f"{baseline_results['Random Forest']['recall_class0']:.4f}",
        f"{baseline_results['Random Forest']['f1_class0']:.4f}",
        f"{baseline_results['Random Forest']['precision_class1']:.4f}",
        f"{baseline_results['Random Forest']['recall_class1']:.4f}",
        f"{baseline_results['Random Forest']['f1_class1']:.4f}"
    ],
    'Tuned': [
        f"{tuned_precision_class0:.4f}",
        f"{tuned_recall_class0:.4f}",
        f"{tuned_f1_class0:.4f}",
        f"{tuned_precision_class1:.4f}",
        f"{tuned_recall_class1:.4f}",
        f"{tuned_f1_class1:.4f}"
    ],
    'Improvement': [
        f"{(tuned_precision_class0 - baseline_results['Random Forest']['precision_class0']):.4f}",
        f"{(tuned_recall_class0 - baseline_results['Random Forest']['recall_class0']):.4f}",
        f"{(tuned_f1_class0 - baseline_results['Random Forest']['f1_class0']):.4f}",
        f"{(tuned_precision_class1 - baseline_results['Random Forest']['precision_class1']):.4f}",
        f"{(tuned_recall_class1 - baseline_results['Random Forest']['recall_class1']):.4f}",
        f"{(tuned_f1_class1 - baseline_results['Random Forest']['f1_class1']):.4f}"
    ]
})

print("\n", per_class_comparison.to_string(index=False))

# ==============================================================================
# DETAILED CLASSIFICATION REPORT
# ==============================================================================
print("\n" + "="*100)
print(" TUNED MODEL - CLASSIFICATION REPORT ")
print("="*100)

print("\n", classification_report(y_test, y_pred_tuned,
                                  target_names=['Not Potable', 'Potable'],
                                  digits=4))

# ==============================================================================
# TOP PARAMETER COMBINATIONS
# ==============================================================================
print("\n" + "="*100)
print(" TOP 10 PARAMETER COMBINATIONS ")
print("="*100)

# Get CV results
cv_results = pd.DataFrame(grid_search.cv_results_)
cv_results = cv_results.sort_values('rank_test_score')

# Select relevant columns
top_params = cv_results[['rank_test_score', 'mean_test_score', 'std_test_score',
                         'param_n_estimators', 'param_max_depth', 'param_min_samples_split',
                         'param_min_samples_leaf', 'param_max_features', 'param_max_samples']].head(10)

print("\n", top_params.to_string(index=False))

# ==============================================================================
# FEATURE IMPORTANCE
# ==============================================================================
print("\n" + "="*100)
print(" FEATURE IMPORTANCE")
print("="*100)

def get_processed_feature_names(preprocessor, input_df_columns):
    output_feature_names = []
    for name, transformer, features_orig in preprocessor.transformers_:
        if transformer == 'passthrough':
            output_feature_names.extend(features_orig)
        elif hasattr(transformer, 'get_feature_names_out'):
            if isinstance(transformer, OneHotEncoder):
                output_feature_names.extend(transformer.get_feature_names_out(features_orig))
            else:
                output_feature_names.extend(features_orig)
        else:
            output_feature_names.extend(features_orig)
    return output_feature_names

if hasattr(best_tuned_model, 'feature_importances_'):
    feature_names_tuned = get_processed_feature_names(preprocessor, X_train_complete.columns)

    tuned_feature_importance = pd.DataFrame({
        'feature': feature_names_tuned,
        'importance': best_tuned_model.feature_importances_
    }).sort_values('importance', ascending=False)

    print(f"\nTop 10 Most Important Features:")
    print(tuned_feature_importance.head(10).to_string(index=False))

    print(f"\nBottom 5 Least Important Features:")
    print(tuned_feature_importance.tail(5).to_string(index=False))

# ==============================================================================
# SAVE TUNED MODEL
# ==============================================================================
print("\n" + "="*100)
print(" SAVING TUNED MODEL ")
print("="*100)

os.makedirs('models', exist_ok=True)

# Save the tuned model
model_filename = f'models/random_forest_tuned_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib'
joblib.dump(best_tuned_model, model_filename)
print(f"\n Tuned model saved to: {model_filename}")

# Save grid search results
grid_results_filename = f'models/grid_search_results_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib'
joblib.dump(grid_search, grid_results_filename)
print(f"Grid search results saved to: {grid_results_filename}")

# Save comparison summary
summary = {
    'baseline_performance': {
        'cv_roc_auc': baseline_cv_roc_auc,
        'test_roc_auc': baseline_test_roc_auc,
        'test_f1': baseline_results['Random Forest']['test_f1']
    },
    'tuned_performance': {
        'cv_roc_auc': grid_search.best_score_,
        'cv_f1': cv_f1_mean,
        'test_roc_auc': tuned_test_roc_auc,
        'test_f1': tuned_test_f1
    },
    'best_params': grid_search.best_params_,
    'improvement': {
        'cv_roc_auc': grid_search.best_score_ - baseline_cv_roc_auc,
        'test_roc_auc': tuned_test_roc_auc - baseline_test_roc_auc,
        'test_f1': tuned_test_f1 - baseline_results['Random Forest']['test_f1']
    }
}

summary_filename = f'models/tuning_summary_{datetime.now().strftime("%Y%m%d_%H%M%S")}.joblib'
joblib.dump(summary, summary_filename)
print(f"Tuning summary saved to: {summary_filename}")

print("\n" + "="*100)
print("HYPERPARAMETER TUNING COMPLETE")
print("="*100)

# Final recommendation
print(f"\nRECOMMENDATION:")
if tuned_test_roc_auc > baseline_test_roc_auc:
    improvement_pct = ((tuned_test_roc_auc - baseline_test_roc_auc) / baseline_test_roc_auc) * 100
    print(f"  Use TUNED model - improved  {improvement_pct:.2f}%")
    print(f"  Test ROC-AUC: {baseline_test_roc_auc:.4f} → {tuned_test_roc_auc:.4f}")
else:
    print(f"  Tuned model tidak menunjukkan improvement")



HYPERPARAMETER TUNING - RANDOM FOREST

Baseline Performance:
  CV ROC-AUC:   0.9527
  Test ROC-AUC: 0.9345

Parameter Grid:
  n_estimators: [150, 200]
  max_depth: [10, 20, None]
  min_samples_split: [2, 5]
  min_samples_leaf: [1, 2]
  max_features: ['sqrt', 'log2']
  max_samples: [0.7, 0.8]

Total parameter combinations: 96
Total fits (5-fold CV): 480
Fitting 5 folds for each of 96 candidates, totalling 480 fits

Grid Search completed in 1216.93s (20.28 minutes)

 TUNING RESULTS 

Best Parameters Found:
  max_depth: None
  max_features: sqrt
  max_samples: 0.8
  min_samples_leaf: 1
  min_samples_split: 5
  n_estimators: 200

Best CV ROC-AUC: 0.9536

Calculating CV F1 score...
  CV F1 (mean ± std): 0.8812 ± 0.0069

Evaluating tuned model on test set...

Tuned Model Performance:
  CV ROC-AUC:     0.9536
  CV F1:          0.8812
  Test Accuracy:  0.8490
  Test Precision: 0.8509
  Test Recall:    0.8490
  Test F1:        0.8488
  Test ROC-AUC:   0.9337
  OOB Score:      0.8793

 BASELINE

#PHASE 6: EVALUATION

In [12]:
# ==============================================================================
# FINAL SUMMARY
# ==============================================================================
print("\n" + "="*100)
# print(" FINAL SUMMARY ".center(100))
print("="*100)

# Check if tuned_results is defined
if 'tuned_results' not in globals():
    print("Error: 'tuned_results' is not defined.")
else:
    # Best tuned model
    best_tuned_model = max(tuned_results.keys(), key=lambda k: tuned_results[k]['test_roc_auc'])
    best_tuned_auc = tuned_results[best_tuned_model]['test_roc_auc']

    print(f"\n BEST TUNED MODEL: {best_tuned_model}")
    print(f"   CV ROC-AUC:     {tuned_results[best_tuned_model]['cv_roc_auc']:.4f}")
    print(f"   CV F1:          {tuned_results[best_tuned_model]['cv_f1']:.4f}")
    print(f"   Test ROC-AUC:   {best_tuned_auc:.4f}")
    print(f"   Test F1:        {tuned_results[best_tuned_model]['test_f1']:.4f}")
    print(f"   Test Accuracy:  {tuned_results[best_tuned_model]['test_accuracy']:.4f}")

    print(f"\n Best Parameters:")
    for param, value in tuned_results[best_tuned_model]['best_params'].items():
        print(f"   {param}: {value}")

    # Average improvement
    baseline_auc_avg = np.mean([baseline_results[m]['test_roc_auc'] for m in tuned_results.keys()])
    tuned_auc_avg = np.mean([tuned_results[m]['test_roc_auc'] for m in tuned_results.keys()])
    improvement_auc = ((tuned_auc_avg - baseline_auc_avg) / baseline_auc_avg) * 100

    baseline_f1_avg = np.mean([baseline_results[m]['test_f1'] for m in tuned_results.keys()])
    tuned_f1_avg = np.mean([tuned_results[m]['test_f1'] for m in tuned_results.keys()])
    improvement_f1 = ((tuned_f1_avg - baseline_f1_avg) / baseline_f1_avg) * 100

    print(f"\n   AVERAGE IMPROVEMENT (All {len(tuned_results)} Models):")
    print(f"\n   ROC-AUC:")
    print(f"      Baseline avg: {baseline_auc_avg:.4f}")
    print(f"      Tuned avg:    {tuned_auc_avg:.4f}")
    print(f"      Improvement:  {improvement_auc:+.2f}%")
    print(f"\n   F1-Score:")
    print(f"      Baseline avg: {baseline_f1_avg:.4f}")
    print(f"      Tuned avg:    {tuned_f1_avg:.4f}")
    print(f"      Improvement:  {improvement_f1:+.2f}%")



 BEST TUNED MODEL: Random Forest
   CV ROC-AUC:     0.9536
   CV F1:          0.8812
   Test ROC-AUC:   0.9337
   Test F1:        0.8488
   Test Accuracy:  0.8490

 Best Parameters:
   max_depth: None
   max_features: sqrt
   max_samples: 0.8
   min_samples_leaf: 1
   min_samples_split: 5
   n_estimators: 200

   AVERAGE IMPROVEMENT (All 1 Models):

   ROC-AUC:
      Baseline avg: 0.9345
      Tuned avg:    0.9337
      Improvement:  -0.09%

   F1-Score:
      Baseline avg: 0.8487
      Tuned avg:    0.8488
      Improvement:  +0.01%


# PHASE 7: DEPLOYMENT

In [13]:
import joblib
from google.colab import files

print("="*80)
print(" SAVING MODEL FOR DEPLOYMENT ".center(80))
print("="*80)

# Save model
best_model = tuned_results['Random Forest']['model']
joblib.dump(best_model, 'random_forest_model.pkl')
print("\n Model saved: random_forest_model.pkl")

# Save preprocessor
joblib.dump(preprocessor, 'preprocessor(2).pkl')
print(" Preprocessor saved: preprocessor(2).pkl")

# Download
print("\n Downloading files to your computer...")
files.download('random_forest_model.pkl')
files.download('preprocessor(2).pkl')


                          SAVING MODEL FOR DEPLOYMENT                           

 Model saved: random_forest_model.pkl
 Preprocessor saved: preprocessor(2).pkl



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>